In [18]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS, OWL

# Load data
df = pd.read_csv("barislontar.csv")  # ganti nama file sesuai kebutuhan

# Setup RDF graph dan namespace
g = Graph()
ONT = Namespace("http://yusupbanyuwangi.org/ontology#")

g.bind("", ONT)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)
g.bind("dcterms", DCTERMS)
g.bind("owl", OWL)

# ---------------------------
# Deklarasi Ontologi
# ---------------------------
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), RDFS.label, Literal("Ontologi Lontar Yusup Banyuwangi", lang="id")))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), DCTERMS.creator, Literal("Kelompok SemWeb")))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), DCTERMS.created, Literal("2025-05-20", datatype=XSD.date)))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), OWL.versionInfo, Literal("1.0")))

# ---------------------------
# Definisi Class
# ---------------------------
classes = {
    "Manuskrip": "Representasi naskah kuno Lontar Yusup.",
    "BarisNaskah": "Satu baris dalam naskah Lontar.",
    "TeksAksara": "Teks dalam aksara asli (Arab Pegon).",
    "Transliterasi": "Teks hasil transliterasi ke Latin.",
    "Terjemahan": "Teks hasil terjemahan ke Bahasa Indonesia.",
    "SistemTulisan": "Jenis sistem tulisan yang digunakan (misal Pegon)."
}

for cls, desc in classes.items():
    cls_uri = ONT[cls]
    g.add((cls_uri, RDF.type, OWL.Class))
    g.add((cls_uri, RDFS.label, Literal(cls.replace("Teks", "Teks ").replace("Sistem", "Sistem "), lang="id")))
    g.add((cls_uri, RDFS.comment, Literal(desc, lang="id")))

# ---------------------------
# Definisi ObjectProperty
# ---------------------------
obj_props = [
    ("isFromManuscript", "BarisNaskah", "Manuskrip", "berasal dari manuskrip"),
    ("mengandungAksara", "BarisNaskah", "TeksAksara", "mengandung aksara"),
    ("hasTransliteration", "BarisNaskah", "Transliterasi", "memiliki transliterasi"),
    ("hasTranslation", "BarisNaskah", "Terjemahan", "memiliki terjemahan"),
    ("ditulisMenggunakan", "Manuskrip", "SistemTulisan", "ditulis menggunakan")
]

for prop, domain, range_, label in obj_props:
    prop_uri = ONT[prop]
    g.add((prop_uri, RDF.type, OWL.ObjectProperty))
    g.add((prop_uri, RDFS.domain, ONT[domain]))
    g.add((prop_uri, RDFS.range, ONT[range_]))
    g.add((prop_uri, RDFS.label, Literal(label, lang="id")))

# ---------------------------
# Definisi DatatypeProperty
# ---------------------------
g.add((RDF.value, RDF.type, OWL.DatatypeProperty))
g.add((RDF.value, RDFS.domain, OWL.Thing))
g.add((RDF.value, RDFS.range, XSD.string))
g.add((RDF.value, RDFS.label, Literal("nilai literal", lang="id")))

datatype_props = [
    ("keBahasa", "Terjemahan", "ke Bahasa"),
    ("dariBahasa", "Terjemahan", "dari Bahasa"),
    ("menggunakanAturan", "Transliterasi", "menggunakan aturan")
]

for prop, domain, label in datatype_props:
    prop_uri = ONT[prop]
    g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
    g.add((prop_uri, RDFS.domain, ONT[domain]))
    g.add((prop_uri, RDFS.range, XSD.string))
    g.add((prop_uri, RDFS.label, Literal(label, lang="id")))

# ---------------------------
# Tambahkan Instance Awal (Manuskrip & Sistem Tulisan)
# ---------------------------
manuskrip_uri = ONT["naskahYusup01"]
g.add((manuskrip_uri, RDF.type, ONT.Manuskrip))
g.add((manuskrip_uri, DCTERMS.title, Literal("Lontar Yusup Banyuwangi", lang="jv")))
g.add((manuskrip_uri, DCTERMS.language, Literal("jv")))
g.add((manuskrip_uri, ONT.ditulisMenggunakan, ONT.AksaraPegon))

g.add((ONT.AksaraPegon, RDF.type, ONT.SistemTulisan))
g.add((ONT.AksaraPegon, RDFS.label, Literal("Arab Pegon", lang="id")))

# ---------------------------
# Tambahkan Instance Data per Baris (sepaket)
# ---------------------------
for index, row in df.iterrows():
    idx = f"{int(row['id_baris']):03d}"
    baris_uri = ONT[f"baris{idx}"]
    aksara_uri = ONT[f"aksara_baris{idx}"]
    translit_uri = ONT[f"translit_baris{idx}"]
    terjemah_uri = ONT[f"terjemah_baris{idx}"]

    # Baris Naskah
    g.add((baris_uri, RDF.type, ONT.BarisNaskah))
    g.add((baris_uri, ONT.isFromManuscript, manuskrip_uri))
    g.add((baris_uri, ONT.mengandungAksara, aksara_uri))
    g.add((baris_uri, ONT.hasTransliteration, translit_uri))
    g.add((baris_uri, ONT.hasTranslation, terjemah_uri))
    g.add((baris_uri, DCTERMS.identifier, Literal(f"YUSUP01-B{idx}")))

    # Teks Aksara
    g.add((aksara_uri, RDF.type, ONT.TeksAksara))
    g.add((aksara_uri, RDF.value, Literal(row['aksara'], lang="urlImage")))

    # Transliterasi
    g.add((translit_uri, RDF.type, ONT.Transliterasi))
    g.add((translit_uri, RDF.value, Literal(row['transliterasi'], lang="jv-Latn")))
    g.add((translit_uri, ONT.menggunakanAturan, Literal("Transliterasi Arab Pegon Standar")))

    # Terjemahan
    g.add((terjemah_uri, RDF.type, ONT.Terjemahan))
    g.add((terjemah_uri, RDF.value, Literal(row['terjemahan'], lang="id")))
    g.add((terjemah_uri, ONT.keBahasa, Literal("id")))
    g.add((terjemah_uri, ONT.dariBahasa, Literal("jv")))

# ---------------------------
# Simpan ke TTL
# ---------------------------
g.serialize("turtle/ttl-konversi.ttl", format="turtle")

<Graph identifier=N9a61ebc255d044c0a992aaac3d0b46d6 (<class 'rdflib.graph.Graph'>)>

In [1]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS, OWL

# ----------- Load data CSV ----------
df = pd.read_csv("barislontar2.csv")  # ganti ke nama file CSV-mu

# ----------- RDF Graph & Namespace ----------
g = Graph()
ONT = Namespace("http://yusupbanyuwangi.org/ontology#")

g.bind("", ONT)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)
g.bind("dcterms", DCTERMS)
g.bind("owl", OWL)

# ----------- Deklarasi Ontologi ----------
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), RDFS.label, Literal("Ontologi Lontar Yusup Banyuwangi", lang="id")))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), DCTERMS.creator, Literal("Kelompok SemWeb")))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), DCTERMS.created, Literal("2025-05-20", datatype=XSD.date)))
g.add((URIRef("http://yusupbanyuwangi.org/ontology"), OWL.versionInfo, Literal("1.0")))

# ----------- Definisi Class ----------
classes = {
    "Manuskrip": "Representasi naskah kuno Lontar Yusup.",
    "BarisNaskah": "Satu baris dalam naskah Lontar.",
    "TeksAksara": "Teks dalam aksara asli (Arab Pegon).",
    "Transliterasi": "Teks hasil transliterasi ke Latin.",
    "Terjemahan": "Teks hasil terjemahan ke Bahasa Indonesia.",
    "SistemTulisan": "Jenis sistem tulisan yang digunakan (misal Pegon)."
}

for cls, desc in classes.items():
    cls_uri = ONT[cls]
    g.add((cls_uri, RDF.type, OWL.Class))
    g.add((cls_uri, RDFS.label, Literal(cls.replace("Teks", "Teks ").replace("Sistem", "Sistem "), lang="id")))
    g.add((cls_uri, RDFS.comment, Literal(desc, lang="id")))

# ----------- Definisi ObjectProperty ----------
obj_props = [
    ("isFromManuscript", "BarisNaskah", "Manuskrip", "berasal dari manuskrip"),
    ("mengandungAksara", "BarisNaskah", "TeksAksara", "mengandung aksara"),
    ("hasTransliteration", "BarisNaskah", "Transliterasi", "memiliki transliterasi"),
    ("hasTranslation", "BarisNaskah", "Terjemahan", "memiliki terjemahan"),
    ("ditulisMenggunakan", "Manuskrip", "SistemTulisan", "ditulis menggunakan")
]

for prop, domain, range_, label in obj_props:
    prop_uri = ONT[prop]
    g.add((prop_uri, RDF.type, OWL.ObjectProperty))
    g.add((prop_uri, RDFS.domain, ONT[domain]))
    g.add((prop_uri, RDFS.range, ONT[range_]))
    g.add((prop_uri, RDFS.label, Literal(label, lang="id")))

# ----------- Definisi DatatypeProperty ----------
g.add((RDF.value, RDF.type, OWL.DatatypeProperty))
g.add((RDF.value, RDFS.domain, OWL.Thing))
g.add((RDF.value, RDFS.range, XSD.string))
g.add((RDF.value, RDFS.label, Literal("nilai literal", lang="id")))

datatype_props = [
    ("keBahasa", "Terjemahan", "ke Bahasa"),
    ("dariBahasa", "Terjemahan", "dari Bahasa"),
    ("menggunakanAturan", "Transliterasi", "menggunakan aturan"),
    # Property baru untuk Unicode Pegon
    ("teksPegon", "TeksAksara", "teks Pegon Unicode"),
]

for prop, domain, label in datatype_props:
    prop_uri = ONT[prop]
    g.add((prop_uri, RDF.type, OWL.DatatypeProperty))
    g.add((prop_uri, RDFS.domain, ONT[domain]))
    g.add((prop_uri, RDFS.range, XSD.string))
    g.add((prop_uri, RDFS.label, Literal(label, lang="id")))

# ----------- Tambahkan Instance Awal (Manuskrip & Sistem Tulisan) ----------
manuskrip_uri = ONT["naskahYusup01"]
g.add((manuskrip_uri, RDF.type, ONT.Manuskrip))
g.add((manuskrip_uri, DCTERMS.title, Literal("Lontar Yusup Banyuwangi", lang="jv")))
g.add((manuskrip_uri, DCTERMS.language, Literal("jv")))
g.add((manuskrip_uri, ONT.ditulisMenggunakan, ONT.AksaraPegon))

g.add((ONT.AksaraPegon, RDF.type, ONT.SistemTulisan))
g.add((ONT.AksaraPegon, RDFS.label, Literal("Arab Pegon", lang="id")))

# ----------- Tambahkan Instance Data per Baris (sepaket) ----------
for index, row in df.iterrows():
    idx = f"{int(row['id_baris']):03d}"
    baris_uri = ONT[f"baris{idx}"]
    aksara_uri = ONT[f"aksara_baris{idx}"]
    translit_uri = ONT[f"translit_baris{idx}"]
    terjemah_uri = ONT[f"terjemah_baris{idx}"]

    # Baris Naskah
    g.add((baris_uri, RDF.type, ONT.BarisNaskah))
    g.add((baris_uri, ONT.isFromManuscript, manuskrip_uri))
    g.add((baris_uri, ONT.mengandungAksara, aksara_uri))
    g.add((baris_uri, ONT.hasTransliteration, translit_uri))
    g.add((baris_uri, ONT.hasTranslation, terjemah_uri))
    g.add((baris_uri, DCTERMS.identifier, Literal(f"YUSUP01-B{idx}")))

    # Teks Aksara (gambar & Unicode Pegon)
    g.add((aksara_uri, RDF.type, ONT.TeksAksara))
    if pd.notnull(row['aksara']):
        g.add((aksara_uri, RDF.value, Literal(row['aksara'], datatype=XSD.anyURI)))
    if pd.notnull(row['teks_pegon']):
        g.add((aksara_uri, ONT.teksPegon, Literal(row['teks_pegon'], lang="ar")))

    # Transliterasi
    g.add((translit_uri, RDF.type, ONT.Transliterasi))
    if pd.notnull(row['transliterasi']):
        g.add((translit_uri, RDF.value, Literal(row['transliterasi'], lang="jv-Latn")))
        g.add((translit_uri, ONT.menggunakanAturan, Literal("Transliterasi Arab Pegon Standar")))

    # Terjemahan
    g.add((terjemah_uri, RDF.type, ONT.Terjemahan))
    if pd.notnull(row['terjemahan']):
        g.add((terjemah_uri, RDF.value, Literal(row['terjemahan'], lang="id")))
        g.add((terjemah_uri, ONT.keBahasa, Literal("id")))
        g.add((terjemah_uri, ONT.dariBahasa, Literal("jv")))

# ----------- Simpan ke TTL ----------
g.serialize("ttl-konversi-pegon.ttl", format="turtle")


<Graph identifier=N9be7d475954642c89cadde63b1e594e3 (<class 'rdflib.graph.Graph'>)>